In [ ]:
import json
import sys
import re
import platform
import os
from winpython import utils
from collections import OrderedDict
from  pip._vendor.packaging.markers import Marker 

pip_ask = ['pip', 'inspect']
pip_res = (utils.exec_run_cmd(pip_ask)+'\n').splitlines()
pip_json= json.loads('\n'.join(pip_res))

def normalize(this):
    return re.sub(r"[-_.]+", "-", this).lower() # from https://peps.python.org/pep-0503/#normalized-names
# create a dict with
#  key = normalised package name
#     string_elements = 'name', 'version', 'summary'
#     requires =  list of dict with 1 level need downward
#             key = package_key requires
#             req_extra = extra branch need = '' or 'extra'
#             req_version = version needed
#             req_marker = additional conditions


replacements = str.maketrans({' ' : '', '[' : '' , ']' : '' , "'" : "" , '"' : ''})

distro = { }
environment = {
            "implementation_name": sys.implementation.name,
            "implementation_version":  "{0.major}.{0.minor}.{0.micro}".format(sys.implementation.version),
            "os_name": os.name,
            "platform_machine": platform.machine(),
            "platform_release": platform.release(),
            "platform_system": platform.system(),
            "platform_version": platform.version(),
            "python_full_version": platform.python_version(),
            "platform_python_implementation": platform.python_implementation(),
            "python_version": ".".join(platform.python_version_tuple()[:2]),
            "sys_platform": sys.platform,
        }

for p in pip_json['installed']:
    meta = p['metadata']
    name = meta['name']
    version = meta['version']
    key = normalize(name)
    if 'summary' in meta:
        summary = meta['summary']
    else:
        summary = ''
    requires = []
    if 'requires_dist' in meta:
        for i in meta['requires_dist']:
            det = (i+";").split(";")
            req_nameextra = normalize((det[0]+' ').split(' ')[0])
            req_key = normalize((req_nameextra+'[').split('[')[0])
            req_key_extra = req_nameextra[len(req_key)+1:].split(']')[0]
            req_version = det[0][len(req_nameextra):].translate(replacements) 
            req_marker = det[1]
  
            # if no test besides extra_requires branch name, remove the 'if':
            #req_if = req_if.replace("extra == '" + req_branch + "'" , "").strip()
            req_add = []
            if req_marker == '':
                req_add = [{'req_key': req_key,'req_version': req_version, 'req_extra':req_key_extra}]
            else:
                # we could fitler on extra = '' like pipdeptrre but let not at this stage
                # if Marker(req_if).evaluate(environment=envi):
                envi = {'extra':'all', **environment}
                req_add = [{'req_key': req_key,'req_version': req_version, 'req_extra':req_key_extra , 'req_marker':req_marker}]
            requires += req_add
    distro[key] = {'name': name, 'version': version, 'summary': summary, 'requires_dist': requires, 'wanted_per' :[]}

# complement distro in reverse mode with 1 level need UPward 'wanted-per'
# get all downwardlinke in 'requires_dist' of each package
# to feed the required packages 'wanted_per' as a reverse dict of dict
#       '' or 'extra' or = dict key requiring the package
#       'extra'= dict required per branch 'extra'
#        contains = 
#             key = package_key wanting it
#             extra branch need = '' always
#             version = version wanted
#             req_marker = additional conditions

envi = {'extra': 'all', **environment}
for p in distro:
    for r in distro[p]['requires_dist']:
        if r['req_key'] in distro:
            if 1==1: #'req_marker' not in r  or Marker(r['req_marker']).evaluate(environment=envi):
                #want_add = [{'req_key': p,'req_version': req_version, 'req_marker':req_marker}]  #req_key_extra
                if 'req_marker' in r:
                    want_add = [{'req_key': p,'req_version': r['req_version'], 'req_marker': r['req_marker']}]  #req_key_extra
                else:
                    want_add = [{'req_key': p,'req_version': r['req_version']}]  #req_key_extra
                #if p not in distro[r['req_key']]['wanted_per']:
                distro[r['req_key']]['wanted_per'] += want_add

    
def pipdownraw(pp, extra='', version_req=''):
    envi = {'extra': extra, **environment}
    #p = normalize(pp)
    pp_list = [normalize(pp),] if not pp=='' else [pp for pp in distro]
    ret_all = []
    for p in sorted(pp_list):
        if p in distro:
            if extra=='':
                ret=[p + '==' + distro[p]['version'] + ' ' +version_req]
            else:
                ret=[p + '['+ extra +']'+ '==' + distro[p]['version'] + ' ' +version_req]
        
            ret2=[]
            for r in distro[p]['requires_dist']:
                if r['req_key'] in distro:
                    # print(r)
                    if 'req_marker' not in r  or Marker(r['req_marker']).evaluate(environment=envi):
                        #ret += [r['req_key'] +' '+  distro[r['req_key']]['version'],
                        #       r['req_version'],                           
                        ret2 +=pipdownraw( r['req_key'] , r['req_extra'], r['req_version'])
            if not ret2==[]:
                 ret_all += ret+[ret2]
            else:
                ret_all = ret
    return ret_all   

def pipupraw(pp, extra='', version_req='', recur=['',]):
    envi = {'extra': extra, **environment}
    pp_list = [normalize(pp),] if not pp=='' else [pp for pp in distro]
    ret_all = []
    for p in pp_list:
        if p in distro:
            if extra=='':
                ret=[p + '==' + distro[p]['version'] + ' ' +version_req]
            else :
                ret=[p + '['+ extra +']'+ '==' + distro[p]['version'] + ' ' +version_req]

            ret2=[]
            if len(recur)> 20: 
                print(recur)
                z = 1/0    
            for r in distro[p]['wanted_per']:
                # print(distro[p]['wanted_per'][r]['req,_version'])
                if r['req_key'] in distro and r['req_key'] not in recur:
                    #print(r)
                    if 'req_marker' not in r:
                        ret2 +=pipupraw( r['req_key'] , '', r['req_version'], recur +[pp])
                    elif Marker(r['req_marker']).evaluate(environment=envi):
                        #ret += [r['req_key'] +' '+  distro[r['req_key']]['version'],
                        #       r['req_version'],                           
                        ret2 +=pipupraw( r['req_key'] , r['req_marker'], r['req_version'], recur +[p])
            if not ret2==[]:
                 ret_all += ret+[ret2]
            else:
                ret_all = ret
    return ret_all                

def pipdown(pp='', extra='', version_req='', indent=5):
    if not pp=='':
        print('\n'.join([l for l in json.dumps(pipdownraw(pp, extra,version_req), indent=indent).split('\n') if len(l.strip())>2]).replace('"',''))
    else:
        for one_pp in sorted(distro):
            # if distro[one_pp]['wanted_per']==[]:  # would require to now if it's in filtered choice
            print('\n'.join([l for l in json.dumps(pipdownraw(one_pp, extra,version_req), indent=indent).split('\n') if len(l.strip())>2]).replace('"',''))

def pipup(pp, extra='', version_req='', indent=5):
    print('\n'.join([l for l in json.dumps(pipupraw(pp, extra,version_req), indent=indent).split('\n') if len(l.strip())>2]).replace('"',''))


In [ ]:
pipdown('pandas')  # what needs pandas per default

In [ ]:
pipdown('pandas', 'test') # what needs pandas[test]

In [ ]:
pipup('datashader') # what is needed per datashader per default

In [ ]:
pipup('datashader','doc') # what is needed per datashader with '[doc]' requirement

In [ ]:
distro['pytz'] # collected data for pytz

In [ ]:
# pipdown() (very long, but works)

In [ ]:
# to inquire on what 'pip inspect' gives for list of pakage list_of_package
list_of_package = ['isort',]: # aiohttp', 'pandas', 'Pandas', 'dask', 'Dask', 'Spyder', 'spyder']:

import json
from winpython import utils
this = 'sqlite-bro'
this_len = len(this)
pip_ask = ['pip', 'inspect']
pip_res = (utils.exec_run_cmd(pip_ask)+'\n').splitlines()
pip_json= json.loads('\n'.join(pip_res))

pip_filter = [l for l in pip_res ]
#print ('*',pip_json['pip_version'],'*')
dist={}
 #for p in pip_json['installed'] # list of 1 json string per package
#print ('!!', pip_filter2[0], '!!')         
for i in range(3):
    print(pip_res[i])
for p in pip_json['installed']:
    meta = p['metadata']
    name = meta['name']
    #print(meta['name'], meta['version']) # not always, meta['summary']) # not always , meta["requires_dist"]
    if "requires_dist" in meta and "[" in meta["requires_dist"]:
          print(meta['name'], meta["requires_dist"])
    if meta['name'] in list_of_package:
          print(meta['name'], meta["requires_dist"])

